In [1]:
from linguist_wrapper import detect

In [2]:
text_c = '''int *twoSum(int *nums, int numsSize, int target, int *returnSize)
{
    int i, j;
    int *ret = calloc(2, sizeof(int));
    for (i = 0; i < numsSize; i++)
    {
        int key = target - nums[i];
        for (j = i + 1; j < numsSize; j++)
            if (nums[j] == key)
            {
                ret[0] = i;
                ret[1] = j;
            }
    }
    *returnSize = 2;
    return ret;
}'''

text_scala = '''package Mathematics

object AbsMin {

  /** Method returns Absolute minimum Element from the list
    *
    * @param listOfElements
    * @return
    */
  def abs: Int => Int = Abs.abs

  def absMin(elements: List[Int]): Int = abs(elements.minBy(x => abs(x)))

}'''

In [3]:
detect(text_c)

C


In [4]:
detect(text_scala)

Scala
